# LionAGI Cookbook

## Chapter 2: Building a Customer Service Bot

In [Chapter 1](ch1_get_started.md), we built a **research assistant** primarily using the `branch.chat()` method. 

That approach was **single-turn**: each call to `chat()` did **not** add messages to the conversation history. Now, we’ll explore **LionAGI’s** architecture and focus on **multi-turn** usage with `branch.communicate()`, which **does** store messages for continuous dialogue.

---

## 1. Architecture & Design Philosophy

LionAGI is deliberately **modular**. When building a “Customer Service” bot, these key pieces unite:

1. **Branch**  
   - Coordinates a single conversation.  
   - Maintains messages, tools, logs, and references to an **iModel**.  
   - We’ll use `communicate()` for multi-turn conversation memory.

2. **iModel**  
   - Represents a configured LLM endpoint (like GPT-4o, sonnet-3.5, etc.).  
   - Manages concurrency, token usage, and request settings.

3. **Action System** (Optional)  
   - Tools exposed to the LLM for real backend operations (e.g., resetting a password, issuing refunds).  
   - The LLM can “function-call” them if it decides it needs those capabilities.

4. **Messages & Conversation**  
   - `branch.communicate()` automatically **stores** user messages + assistant replies in order, supporting multi-turn memory.  
   - `branch.chat()`, by contrast, is typically a single-turn approach (no automatic message storage).

5. **Logs & Session** (Advanced)  
   - You can attach a `LogManager` or use a `Session` to handle multiple conversation branches and cross-branch communication.  
   - For now, we’ll stick to a single Branch for a single conversation flow.

### High-Level Flow for Customer Service
1. **User** asks a question or describes a problem.  
2. **Branch** uses an **iModel** to generate a reply, storing both the user inquiry and the assistant response in the conversation.  
3. If a function call is needed (e.g., “reset_password”), the Branch’s tool system handles it.  
4. The conversation continues seamlessly: new user messages have context from previous messages because we used `communicate()`.

---

## 2. Example: Building a Basic Customer Service Branch

Here’s a **LionAGI** approach for multi-turn conversation. Let’s define a simple Tool, an iModel, and a Branch that uses `communicate()`.


In [2]:
from lionagi import iModel, Branch

customer_service_model = iModel(
    provider="openai",
    model="gpt-4o-mini",
    temperature=0.7,
    # concurrency/rate-limits can be set if needed
)

service_branch = Branch(
    name="CustomerService",
    system="""You are a polite customer service agent.
    - Greet the user.
    - Provide helpful info or next steps.
    - Escalate if the issue is out of scope.""",
    chat_model=customer_service_model,
)

In [3]:
async def handle_inquiry(user_id: str, user_message: str) -> str:
    """
    Takes the user's message and returns an AI response,
    preserving conversation history for follow-up questions.
    """
    # Provide context if needed (e.g., user_id)
    response = await service_branch.communicate(
        instruction=user_message,
        context={"user_id": user_id},
    )
    return response

In [4]:
async def customer_service_demo():
    # 1) First inquiry
    resp1 = await handle_inquiry("User123", "Hi, I forgot my password.")
    print("Assistant says:", resp1)

    # 2) Follow-up inquiry (the user is still locked out)
    resp2 = await handle_inquiry("User123", "I'm still locked out. Help!")
    print("Assistant says:", resp2)

    # 3) Another question, context is still remembered
    resp3 = await handle_inquiry(
        "User123", "Thanks! How do I change my billing address next?"
    )
    print("Assistant says:", resp3)

In [5]:
await customer_service_demo()

Assistant says: To reset your password, please follow these steps:

1. Go to the login page of the platform.
2. Click on the "Forgot Password?" link.
3. Enter your user ID (User123) or the email associated with your account.
4. Check your email for a password reset link.
5. Follow the instructions in the email to create a new password.

If you encounter any issues, feel free to reach out to customer support for further assistance.
Assistant says: # Assistant Response

**Response**:
I'm sorry to hear that you're still locked out. Here are some additional steps you can take:

1. **Check Your Spam/Junk Folder**: Sometimes, the password reset email might end up in the spam or junk folder. Make sure to check there.

2. **Resend the Password Reset Email**: Go back to the login page and click on "Forgot Password?" to request a new password reset email.

3. **Contact Customer Support**: If you’re still having trouble, please reach out to customer support directly. Provide them with your user I

In [6]:
df = service_branch.logs.to_df()
df.head()

,id,created_at,metadata,content
0,d75851ef-96af-45ad-aed9-fea90b0411cd,2025-01-15 16:01:09.180989,{'lion_class': 'lionagi.protocols.generic.log....,"{'id': '64471e93-5da3-4845-a798-983afc1ce172',..."
1,c81bf60d-df85-4f04-9a87-1b1d95477511,2025-01-15 16:01:12.960887,{'lion_class': 'lionagi.protocols.generic.log....,"{'id': '0476c0b2-a2db-4cbc-aaad-4e41e3f4544d',..."
2,bbc10810-ca4a-469f-bbaa-025e3591bdf3,2025-01-15 16:01:16.615704,{'lion_class': 'lionagi.protocols.generic.log....,"{'id': '0b4b4331-a4f7-4ba4-a16e-29114c836c55',..."


In [ ]:
# you can save the logs to a file
# service_branch.dump_logs(clear=False, persist_path="customer_service_logs.json")

# csv format is also supported, clear=True will clear the logs after saving
# service_branch.dump_logs(clear=True, persist_path="customer_service_logs.csv")

In [7]:
# this will produce a formatted dataframe of messages in the conversation
df1 = service_branch.to_df()
df1.head()

,created_at,role,content,id,sender,recipient,metadata
0,2025-01-15 16:01:04.905925,system,{'system_message': 'You are a polite customer ...,905391fd-5d43-46c6-9393-024cdd18ee56,system,068bd513-e35c-4196-8898-0bc13de2fccc,{'lion_class': 'lionagi.protocols.messages.sys...
1,2025-01-15 16:01:04.918373,user,"{'context': [{'user_id': 'User123'}], 'instruc...",f2bf273d-a227-47c5-ae75-a2f4bfeb25de,user,068bd513-e35c-4196-8898-0bc13de2fccc,{'lion_class': 'lionagi.protocols.messages.ins...
2,2025-01-15 16:01:09.181037,assistant,{'assistant_response': 'To reset your password...,deeaeb7f-1e19-46f1-9baa-d593b5f74926,068bd513-e35c-4196-8898-0bc13de2fccc,user,{'model_response': {'id': 'chatcmpl-Aq4nbpPq3m...
3,2025-01-15 16:01:09.181259,user,"{'context': [{'user_id': 'User123'}], 'instruc...",6230928c-4295-47fa-b7d4-950b3ff1d8c8,user,068bd513-e35c-4196-8898-0bc13de2fccc,{'lion_class': 'lionagi.protocols.messages.ins...
4,2025-01-15 16:01:12.960932,assistant,{'assistant_response': '# Assistant Response ...,44cf14a2-d102-4009-856f-4d58901810d8,068bd513-e35c-4196-8898-0bc13de2fccc,user,{'model_response': {'id': 'chatcmpl-Aq4ndE6SP7...


In [8]:
# whereas below will convert all messages data in the conversation into dataframe
df2 = service_branch.messages.to_df()
df2.head()

,id,created_at,metadata,content,embedding,role,template,sender,recipient
0,905391fd-5d43-46c6-9393-024cdd18ee56,2025-01-15 16:01:04.905925,{'lion_class': 'lionagi.protocols.messages.sys...,{'system_message': 'You are a polite customer ...,None,system,None,system,068bd513-e35c-4196-8898-0bc13de2fccc
1,f2bf273d-a227-47c5-ae75-a2f4bfeb25de,2025-01-15 16:01:04.918373,{'lion_class': 'lionagi.protocols.messages.ins...,"{'context': [{'user_id': 'User123'}], 'instruc...",None,user,None,user,068bd513-e35c-4196-8898-0bc13de2fccc
2,deeaeb7f-1e19-46f1-9baa-d593b5f74926,2025-01-15 16:01:09.181037,{'model_response': {'id': 'chatcmpl-Aq4nbpPq3m...,{'assistant_response': 'To reset your password...,None,assistant,None,068bd513-e35c-4196-8898-0bc13de2fccc,user
3,6230928c-4295-47fa-b7d4-950b3ff1d8c8,2025-01-15 16:01:09.181259,{'lion_class': 'lionagi.protocols.messages.ins...,"{'context': [{'user_id': 'User123'}], 'instruc...",None,user,None,user,068bd513-e35c-4196-8898-0bc13de2fccc
4,44cf14a2-d102-4009-856f-4d58901810d8,2025-01-15 16:01:12.960932,{'model_response': {'id': 'chatcmpl-Aq4ndE6SP7...,{'assistant_response': '# Assistant Response ...,None,assistant,None,068bd513-e35c-4196-8898-0bc13de2fccc,user


In [9]:
from lionagi import Branch, iModel

# 1) Model configuration
cs_model = iModel(provider="openai", model="gpt-4o", temperature=0.7)

# 2) Branch creation
cs_branch = Branch(
    name="CustomerSupport",
    system="You are a friendly agent with memory of previous messages.",
    chat_model=cs_model,
)


# 3) Multi-turn conversation
async def quick_demo():
    # First user message
    first_resp = await cs_branch.communicate("Hi, I need help with my account")
    print("Assistant:", first_resp)

    # Follow-up user message
    second_resp = await cs_branch.communicate(
        "I'm also locked out of billing."
    )
    print("Assistant:", second_resp)

In [10]:
await quick_demo()

Assistant: Hello! I'd be happy to help you with your account. Could you please provide more details about the issue you're experiencing or the type of assistance you need?
Assistant: I'm sorry to hear that you're locked out of your billing account. Here are some general steps you can take to resolve this issue:

1. **Password Reset**: If you’re unable to access your account due to a forgotten password, try using the "Forgot Password" feature on the login page to reset your password.

2. **Account Verification**: Check your email for any account verification requests or alerts that might need your attention.

3. **Contact Support**: Reach out to the customer support team of the service or platform you’re using. They can provide specific assistance for billing-related issues and account access.

4. **Check for Account Holds**: Sometimes accounts are locked due to billing issues or security concerns. Ensure there are no holds or flags on your account by checking any recent emails or notif